Creating a Unigram Tokenizer on the version 6 of the corpora
----------------------------------
We added new sentences to extracted sentences from the book <i style="color: cyan">Grammaire de Wolof Moderne</i> by Pathé Diagne plus the original corpora to obtain the sixth version of it.

The process is almost the same as in [processing_4](text_processing4.ipynb) excepted that we will create another custom dataset for the custom transformer model and identify with a box plot the range of the maximum length of the sequences in order to tune the `max_len` parameter provided to the custom dataset.

Let us import the necessary libraries.

In [1]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for plotting the box plot of the sequence lengths
import plotly.express as px

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `Bart` model understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [2]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/corpora_v6.csv")

# initialize a batch size
BATCH_SIZE = 400

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french'].to_list() 
    
    wolof_sents = sentences['wolof'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [3]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [4]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [5]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

In [6]:
special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>']

In [7]:
trainer = trainers.UnigramTrainer(special_tokens=special_tokens, unk_token = "<unk>", vocab_size=20000) # let us take the default vocab size

#### Train the tokenizer

In [8]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [9]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 9483


#### Initialize the decoder

In [10]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [11]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/bart_tokenizers/tokenizer_v5.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [12]:
random.seed(200)

french_sentence = random.choice(sentences['french']) 

wolof_sentence = random.choice(sentences['wolof']) 

# For the Bart
french_sentence = '<s>' + french_sentence + "</s>"
wolof_sentence = '<s>' + wolof_sentence + "</s>"


In [13]:
# print the french sentence
french_sentence

"<s>Les fourmis, à Ogoja, étaient des insectes monstrueux de la variété exectoïde, qui creusaient leurs nids à dix mètres de profondeur sous la pelouse du jardin, où devaient vivre des centaines de milliers d'individus.</s>"

In [14]:
# print the wolof sentence
wolof_sentence

'<s>A ngiy dóor dàqeek jànt bi</s>'

In [15]:
french_encoding = tokenizer.encode(french_sentence)

print("French tokens")
print(french_encoding.tokens)

print("French ids")
print(french_encoding.ids)

French tokens
['<s>', '▁Les', '▁fourmis', ',', '▁à', '▁Ogoja', ',', '▁étai', 'ent', '▁des', '▁insectes', '▁m', 'onstru', 'eux', '▁de', '▁la', '▁vari', 'été', '▁ex', 'ecto', 'ï', 'de', ',', '▁qui', '▁creu', 'saient', '▁', 'leurs', '▁ni', 'd', 's', '▁à', '▁di', 'x', '▁mètres', '▁de', '▁profondeur', '▁sous', '▁la', '▁pelouse', '▁du', '▁jardin', ',', '▁où', '▁de', 'vaient', '▁vivre', '▁des', '▁cent', 'aines', '▁de', '▁mill', 'iers', "▁d'", 'individus', '.', '</s>']
French ids
[0, 162, 797, 5, 20, 568, 5, 756, 95, 18, 1522, 219, 8401, 379, 8, 10, 2668, 1933, 1702, 6378, 2671, 382, 5, 19, 7586, 6136, 6, 183, 24, 187, 9, 20, 35, 227, 4154, 8, 5322, 467, 10, 3663, 33, 905, 5, 124, 8, 730, 1779, 18, 3542, 4531, 8, 2863, 771, 189, 4834, 7, 2]


In [16]:
wolof_encoding = tokenizer.encode(wolof_sentence)

print("Wolof tokens")
print(wolof_encoding.tokens)

print("Wolof ids")
print(wolof_encoding.ids)

Wolof tokens
['<s>', '▁A', '▁ngi', 'y', '▁dóor', '▁dàqee', 'k', '▁jàn', 't', '▁bi', '</s>']
Wolof ids
[0, 201, 62, 13, 680, 4225, 106, 2021, 40, 29, 2]


#### Creating the custom dataset for the new sentences

We have two possibilities to use the tokenizer for fine-tuning a Bart model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [17]:
# from transformers import PreTrainedTokenizerFast

# wrapped_tokenizer1 = PreTrainedTokenizerFast(
#     tokenizer_object=tokenizer,
#     bos_token="<s>",
#     eos_token="</s>",
#     unk_token="<unk>",
#     pad_token="<pad>",
#     cls_token="<cls>",
#     sep_token="<sep>",
#     mask_token="<mask>",
#     padding_side="left",
# )

- Or give directly the tokenizer to the `BartTokenizerFast` class.

In [18]:
from transformers import BartTokenizerFast

wrapped_tokenizer1 = BartTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token = "<s>"
)

Let us give them the sentences that we use as example. 

In [19]:
fr_encoding = wrapped_tokenizer1(french_sentence, max_length=15, padding='max_length', truncation=True)

fr_encoding

{'input_ids': [0, 4808, 112, 797, 5, 20, 568, 5, 756, 95, 18, 1522, 219, 8401, 379], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
wf_encoding = wrapped_tokenizer1(wolof_sentence, max_length=15, padding='max_length', truncation=True)

wf_encoding

{'input_ids': [0, 2781, 62, 13, 680, 4225, 106, 2021, 40, 29, 2, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]}

Let us decode the wolof sentence.

In [21]:
wrapped_tokenizer1.decode(wf_encoding.input_ids, skip_special_tokens=True)

'A ngiy dóor dàqeek jànt bi'

We can see that the `BartTokenizer` add padding to the right side of the sequence while the `PretrainedTokenizer` add the padding to the left side. We can change the padding side from the settings. But, for the next steps, let us directly use the `BartTokenizer`.

**Note that we can augment the sentences when generating them like we did when using the `GPT2 model`.** See the following notebook, [augmentation](text_augmentation.ipynb), for discussion on the augmentation method that we will use. And for a more clear explanation of the augmentation methods in NLP tasks and training, look at the following article from the web [augment_or_not](https://direct.mit.edu/coli/article/48/1/5/108844/To-Augment-or-Not-to-Augment-A-Comparative-Study).

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation. We must for that trace the box plot of the lengths and identify the range in which we will sample the max length of the sequences.

In [22]:
length = []

for sent in sentences['french'].to_list() + sentences['wolof'].to_list():
    
    len_ids = len(wrapped_tokenizer1(sent).input_ids)
    
    length.append(len_ids)

        

In [23]:
fig = px.box(length, template="plotly_dark", labels=dict(x="Length of the sentences", y="Number of sentences"), color_discrete_sequence=['indianred'])

fig.show()

The upper fence is of **51** and the max length is equal to **278**. Then we will test any value between the two. 

But considering the augmentation we can obtain more than the value that we will take because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. We will add to the max length the fifth of it. 

In [24]:
%run wolof-translate/wolof_translate/data/dataset_v4.py

Let us generate some data with their masks and decode the labels.

In [25]:
# Initialize our custom dataset
dataset = SentenceDataset("data/extractions/new_data/corpora_v6.csv", wrapped_tokenizer1, truncation=True, add_bos_token=True)

In [26]:
generator = torch.manual_seed(5)
input_ids, input_mask, labels, _ = next(iter(DataLoader(dataset, 10, shuffle=True, generator=generator))) # generate 10 sentences with shuffling

Let us print the input ids.

In [27]:
input_ids

tensor([[   0, 7015,   37,  668,    5,   37,  668,   31,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   0, 6832,   86,   44,    7,    2,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   0, 9363,   77,  358,    9,  203,  589, 1237,    7,    2,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,

In [28]:
input_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [29]:
labels

tensor([[   0, 9465,  928,   10,  158,  373,   10,   31,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   0, 8652,   14,  277,  178,    7,    2,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   0, 9240,   91,   59,  281,   48,  302,    7,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,

Let us decode the labels.

In [30]:
dataset.decode(labels)

['Kenn la kenn doŋŋ la!',
 'Mi ŋgi fi.',
 'Gisoon naa xar yi yépp.',
 'Wax jépp',
 'Gis ŋga nit ki woon?',
 'Yaw mile mi fii!',
 'Góor gi moo la gisul',
 'Fa la jëkkee xam lu mbégte tekki, fa la jëkkee gis dex yu mag a mag ya, muy Niseer di Bénuwe. Tangori Kamerun yi tamit, jamono jooju la leen njëkk a teg bët.',
 'Gis naa gaynde.',
 'Sama yaay di wësin bees, war a daw laquji ca bëj-saalumu Frãs – mooy fa Almaañ bañoon a yilif. Waaye Pari la ko jaare.']